In [1]:
import numpy as np

In [3]:
class PSO_Exam:
    def __init__(self, n_particles=50, n_dim=10, max_evals=100000):
        self.n_particles = n_particles
        self.n_dim = n_dim
        self.max_evals = max_evals
        
        # --- 1. INICIALIZACIÓN ---
        # Rango [-5.12, 5.12]
        self.X = np.random.uniform(-5.12, 5.12, (n_particles, n_dim))
        # Velocidad inicial pequeña (ceros o random pequeño)
        self.V = np.random.randn(n_particles, n_dim) * 0.1
        
        # --- 2. MEMORIA (pBest y gBest) ---
        # Mejor posición personal (pBest): Al principio es la actual
        self.pBest_pos = self.X.copy()
        # Mejor nota personal: Al principio es la actual
        self.pBest_val = np.array([self.get_fitness(ind) for ind in self.X])
        
        # Mejor global (gBest): El mejor de los personales
        best_idx = np.argmin(self.pBest_val)
        self.gBest_pos = self.pBest_pos[best_idx].copy()
        self.gBest_val = self.pBest_val[best_idx]
        
        # Hiperparámetros del enunciado
        self.w = 0.7  # Inercia
        self.c1 = 1.5 # Cognitivo
        self.c2 = 1.5 # Social

    def get_fitness(self, particle):
        """
        Función Sphere: Suma de cuadrados.
        Recibe un array (10,) y devuelve un float.
        """
        return np.sum(particle**2)

    def update_particles(self):
        """
        El Motor Físico: Mueve todo el enjambre a la vez usando matrices.
        """
        # Generamos aleatorios (r1, r2) para TODAS las coordenadas de golpe
        r1 = np.random.rand(self.n_particles, self.n_dim)
        r2 = np.random.rand(self.n_particles, self.n_dim)
        
        # --- FÓRMULA DE VELOCIDAD (Vectorizada) ---
        # v_new = w*v + c1*r1*(pBest - X) + c2*r2*(gBest - X)
        inertia = self.w * self.V
        cognitive = self.c1 * r1 * (self.pBest_pos - self.X)
        social = self.c2 * r2 * (self.gBest_pos - self.X) # Numpy resta vector gBest a cada fila de X
        
        self.V = inertia + cognitive + social
        
        # --- FÓRMULA DE POSICIÓN ---
        self.X = self.X + self.V
        
        # Opcional: Limitar al rango (rebotar o clippear) para no salir del mapa
        self.X = np.clip(self.X, -5.12, 5.12)

    def run(self):
        evals = 0
        iteration = 0
        
        # Como evaluamos 50 partículas por ciclo, avanzamos de 50 en 50
        while evals < self.max_evals:
            iteration += 1
            
            # 1. MOVER ENJAMBRE
            self.update_particles()
            
            # 2. EVALUAR Y ACTUALIZAR MEMORIA
            # Calculamos fitness de las nuevas posiciones
            # (Vectorizado: aplicamos la función a cada fila)
            current_fitness_values = np.array([self.get_fitness(p) for p in self.X])
            evals += self.n_particles
            
            # 3. ACTUALIZAR pBest (Personal)
            # Buscamos dónde hemos mejorado (máscara booleana)
            improved_indices = current_fitness_values < self.pBest_val
            
            # Actualizamos solo donde hubo mejora
            self.pBest_pos[improved_indices] = self.X[improved_indices]
            self.pBest_val[improved_indices] = current_fitness_values[improved_indices]
            
            # 4. ACTUALIZAR gBest (Global)
            # Miramos si alguien de la nueva generación ha superado el récord mundial
            min_val_iter = np.min(self.pBest_val)
            min_idx_iter = np.argmin(self.pBest_val)
            
            if min_val_iter < self.gBest_val:
                self.gBest_val = min_val_iter
                self.gBest_pos = self.pBest_pos[min_idx_iter].copy()
                print(f"Iter {iteration} (Evals {evals}): Nuevo Récord Global -> {self.gBest_val:.10f}")
                
        return self.gBest_val

# --- TEST DEL EXAMEN ---
pso_solver = PSO_Exam()
resultado = pso_solver.run()
print(f"\nResultado final (debe ser cercano a 0): {resultado}")

Iter 1 (Evals 50): Nuevo Récord Global -> 14.9036228420
Iter 2 (Evals 100): Nuevo Récord Global -> 13.0658023620
Iter 3 (Evals 150): Nuevo Récord Global -> 11.2388340435
Iter 5 (Evals 250): Nuevo Récord Global -> 7.6774060288
Iter 6 (Evals 300): Nuevo Récord Global -> 6.0161215888
Iter 7 (Evals 350): Nuevo Récord Global -> 3.9509996625
Iter 8 (Evals 400): Nuevo Récord Global -> 1.5446306368
Iter 9 (Evals 450): Nuevo Récord Global -> 1.5042438366
Iter 11 (Evals 550): Nuevo Récord Global -> 1.3668676266
Iter 13 (Evals 650): Nuevo Récord Global -> 1.3270299499
Iter 15 (Evals 750): Nuevo Récord Global -> 0.9677508472
Iter 16 (Evals 800): Nuevo Récord Global -> 0.6983232995
Iter 21 (Evals 1050): Nuevo Récord Global -> 0.5651765305
Iter 22 (Evals 1100): Nuevo Récord Global -> 0.4088174951
Iter 23 (Evals 1150): Nuevo Récord Global -> 0.1967915343
Iter 24 (Evals 1200): Nuevo Récord Global -> 0.1795922476
Iter 28 (Evals 1400): Nuevo Récord Global -> 0.1171850144
Iter 30 (Evals 1500): Nuevo Réco